## FAISS

FAISS (Facebook AI Similarity Search) is a library that allows developers to quickly search for embeddings of multimedia documents that are similar to each other. It solves limitations of traditional query search engines that are optimized for hash-based searches, and provides more scalable similarity search functions.

But it can be also used as vector database if your requirement is smaller. It is kind of like a light weight in memory vector database.

There are lot of vector databases like Pine, Chroma, Milvus. But we will use FAISS as our requirement is smaller

### What we will do with this?

Once we will have set of chunks, which we have created using recursive text splitter, we will convert those into embeddings.

**Embedding conversion is a must step.**

And we will store these embeddings in a vector database.

So here for our project we will FAISS index which will do a faster search on our vectors.

*Example: -*

Suppose you have created 1 million vectors out of your chunks. **FAISS** will efficiently perform a search for a given vector. And it will tell you out of those 1 million vector, how many of those vectors are similiar. 

In [2]:
#Install Packages
!pip install faiss-cpu

   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.5 MB 1.3 MB/s eta 0:00:12
   ---------------------------------------- 0.0/14.5 MB 487.6 kB/s eta 0:00:30
   ---------------------------------------- 0.1/14.5 MB 871.5 kB/s eta 0:00:17
   ---------------------------------------- 0.1/14.5 MB 717.5 kB/s eta 0:00:21
   ---------------------------------------- 0.1/14.5 MB 652.5 kB/s eta 0:00:23
   ---------------------------------------- 0.2/14.5 MB 573.4 kB/s eta 0:00:26
   ---------------------------------------- 0.2/14.5 MB 573.4 kB/s eta 0:00:26
   ---------------------------------------- 0.2/14.5 MB 573.4 kB/s eta 0:00:26
   ---------------------------------------- 0.2/14.5 MB 573.4 kB/s eta 0:00:26
   ---------------------------------------- 0.2/14.5 MB 573.4 kB/s eta 0:00:26
   ---------------------------------------- 0.2/14.5 MB 573.4 kB/s eta 0:00:26
   ---------------------------------------- 0.2/14.5 MB 573.4 k

In [3]:
!pip install sentence-transformers

     ---------------------------------------- 0.0/134.8 kB ? eta -:--:--
     -------- ------------------------------ 30.7/134.8 kB 1.4 MB/s eta 0:00:01
     ----------- ------------------------- 41.0/134.8 kB 653.6 kB/s eta 0:00:01
     ----------- ------------------------- 41.0/134.8 kB 653.6 kB/s eta 0:00:01
     ------------------------------------ 134.8/134.8 kB 890.0 kB/s eta 0:00:00
     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.4 kB ? eta -:--:--
     -------------------------------------- 60.4/60.4 kB 796.9 kB/s eta 0:00:00
  Using cached pillow-10.2.0-cp310-cp310-win_amd64.whl.metadata (9.9 kB)
  Using cached fsspec-2024.3.1-py3-none-any.whl.metadata (6.8 kB)
   ---------------------------------------- 0.0/163.3 kB ? eta -:--:--
   --------------------------- ------------ 112.6/163.3 kB 3.3 MB/s eta 0:00:01
   ----------------------------------- ---- 143.4/163.3 kB 1.4 MB/s eta 0:00:01
   ---------

In [5]:
!pip install pandas

  Using cached pandas-2.2.1-cp310-cp310-win_amd64.whl.metadata (19 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.1-cp310-cp310-win_amd64.whl (11.6 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)


In [6]:
# import necessary libraries
import pandas as pd

In [7]:
pd.set_option('display.max_colwidth', 100)

In [8]:
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [9]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


## Step 1 : Create source embeddings for the text column

In [10]:
from sentence_transformers import SentenceTransformer

C:\Users\Shikha Pandey\Documents\github\LLM_langchain_Tutorial\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
encoder = SentenceTransformer("all-mpnet-base-v2")

C:\Users\Shikha Pandey\Documents\github\LLM_langchain_Tutorial\venv\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Shikha Pandey\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [12]:
vectors = encoder.encode(df.text)

In [13]:
vectors.shape

(8, 768)

In [14]:
vectors

array([[-0.00247396,  0.03626723, -0.05290459, ..., -0.09152357,
        -0.03970001, -0.04330489],
       [-0.03357268,  0.00980517, -0.03250129, ..., -0.05165466,
         0.02245888, -0.03156181],
       [-0.01865322, -0.04051315, -0.01235386, ...,  0.00610586,
        -0.07179644,  0.02773852],
       ...,
       [-0.00066459,  0.04252128, -0.05645508, ...,  0.01315471,
        -0.03183568, -0.04357662],
       [-0.03317154,  0.03252454, -0.02484838, ...,  0.0117442 ,
         0.05747125,  0.00571021],
       [-0.00166395,  0.00413827, -0.04597083, ...,  0.02008528,
         0.05656242, -0.00161596]], dtype=float32)

In [15]:
dim = vectors.shape[1]
dim

768

## Step 2 : Build a FAISS Index for vectors

In [16]:
import faiss

this line of code initializes a flat index structure for performing similarity search using the L2 distance metric, and it's configured to work with vectors of a specified dimensionality (dim).

In [17]:
index = faiss.IndexFlatL2(dim)

### Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [18]:
index.add(vectors)

*This line adds vectors to the index. It assumes that vectors is a collection of vectors that you want to add to the index for later similarity search. This step is crucial because it populates the index with data to be searched.*

In [19]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000001BE082D57D0> >

### Step 4 : Encode search text using same encorder and normalize the output vector

In [30]:
search_query = "I want to buy a polo t-shirt"
#search_query = "looking for places to visit during the holidays"
#search_query = "An apple a day keeps the doctor away"

*this line defines a search query. It's a string representing the query you want to search for similarities with in the indexed vectors.*

In [31]:
vec = encoder.encode(search_query)
vec.shape

(768,)

*This line encodes the search query into a vector representation using some encoder. It seems like you're using an encoder to convert the search query into a numerical vector representation. This vector will be used for similarity search against the indexed vectors.*

In [35]:
import numpy as np

In [36]:
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

*This line prints the shape of the svec array. It gives you information about the dimensions of the array. In this case, it will tell you the shape of the vector representation of the search query.*

In [38]:
#faiss.normalize_L2(svec)

### Step 5: Search for similar vector in the FAISS index created

In [40]:
distances, I = index.search(svec, k=2)
distances

array([[1.3844836, 1.4039094]], dtype=float32)

In [41]:
I

array([[3, 2]], dtype=int64)

In [42]:
I.tolist()

[[3, 2]]

In [43]:
row_indices = I.tolist()[0]
row_indices

[3, 2]

In [44]:
df.loc[row_indices]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [45]:
search_query

'I want to buy a polo t-shirt'